<a href="https://colab.research.google.com/github/grniemeyer/IANNwTF_hw2/blob/main/PlayingWithMnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Data prep

import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np

# tensorflow? = (data) pipeline, factory
# tensorflow tensor like array


ds = tfds.load("mnist", split="train")

# remember order for exam!!
ds = ds.map(lambda feature_dict: (feature_dict("image"), feature_dict("label"))) # ectract images and features
ds = ds.map(lambda image, label: (tf.reshape(image, (-1,)), label)) # flatten images, pass on labels
ds = ds.map(lambda imaga, label: ((tf.cast(image, tf.float32/128)-1, label))) # normalize images and pass on
ds = ds.map(lambda image, label: (image, tf.one_hot(label, depth=10))) # one hottify
ds = ds.shuffle(1024).batch(4) # create small buffer(9 pieces of choc bzw. 1024) in between for shuffling then batch
ds = ds.prefetch(4) # parralelisation: 2nd buffer that has always shuffled (prefetched) batches availabe. Shuffling & prefetching takes lot of memory


for x, y in ds.take(1):
  print(x, y)

TypeError: ignored

In [ ]:
# Model Creation via Subclassing from tf.keras.Model

#inherit from keras.Model
class MLP_Model(self, layer_size):
  def __init__(self, layer_size, output_size=10):
    super().__init__()
    self.layers = []
    #layer_size e.g.[256, 256]

    for layer_size in layer_sizes:
      new_layer = tf.keras.layers.Dense(units=layer_size, activation="sigmoid")
      self.layers.append(new_layer)
    self.output_layer = tf.keras.layers.Dense(units=output_size, activation="softmax") # output layer ist deshalb specified seperately bc we need different activation function

  def call(self, x):
    for layer in self.layers:
      x = layer(x) # call layer on input
    y = self.output.layer(x)
    return y

In [ ]:
# Training

EPOCHS = 10

model = MLP_Model(layer_sizes=[256, 256])
cce = tf.keras.CategoricalCrossEntropy()
optimizer = tf.keras.optimizers.legacy.SGD() # stochastic gradient as optimizer
ds = ds # does nothing, just as reminder from above

for epoch in range(EPOCHS):
  losses = []

  for x, target in ds:
    with tf.GradientTape() as tape: # context manager
      pred = model(x)
      loss = cce(target, pred)
    gradients = tape.gradient(loss, model.varibles) # calculate gradient outside of GradientTape, otherwis it calculates gradient of gradient (memory!!, rare exceptions)
    optimizer.appy_gradients(zip(gradients, model.variables))
    losses.append(loss.numpy())
  print(np.mean(losses))

### TF Session 20.11.23 - custom gradient

In [13]:
# custom gradient
import tensorflow as tf
import numpy as np


sample_input = tf.random.uniform(shape = (4,2), dtype=np.float32)

# @ - indicates a decorater function

#tf.custom_gradient()


# writing own sigmoid

@tf.custom_gradient
def my_sigmoid(x):
  sigmoid_res = 1/1+tf.exp(-x)
  def grad(upstream):
    diag_of_jacobian = sigmoid_res*(1-sigmoid_res)
    downstream = upstream*diag_of_jacobian
  return sigmoid_res, grad

In [9]:
# writing own ReLu

# hint: use with tf.where

def my_relu(x):
  relus = tf.where(x>0, x, tf.zeros_like(x))

  def grad(upstream):
    d_dx = tf.where(x>0, tf.ones_like(x), tf.zeros_like(x))
    downstream = upstream*d_dx
    return downstream
  return relus, grad

In [15]:
# explain gradientTape

sample_input = tf.Variable(initial_value=sample_input)
with tf.GradientTape(persistent = True) as tape:
  my_relu_act = my_relu(sample_input)
#  tf_relu_act = tf.nn.relu(sample_input) # common tf relu for comparison (however not possible to compute together, too much memory asked, so not possible or tell to be "persistent", buuut can crash your memory) 1.0
#grads_tf_relu = tape.gradient(tf_relu_act, [sample_input]) # 2.0
grads_my_relu = tape.gradient(my_relu_act, [sample_input])
print(grads_my_relu)

TypeError: ignored

In [18]:
res = []
sample_input = tf.Variable(initial_value=tf.random.uniform(shape=(4,3)))

@tf.function # saves time??, however only for tensorflow code, NO classical python code (print(), .append(), etc)
def some_tf_loop():
  for step in tf.range(10):
    inputs = sample_input[step,:,:]
  with tf.GradientTape(persistent = True) as tape:
    my_relu_act = my_relu(sample_input)
    tf_relu_act = tf.nn.relu(sample_input)
  grads_tf_relu = tape.gradient(tf_relu_act, [sample_input])
  grads_my_relu = tape.gradient(my_relu_act, [sample_input])
  del tape

print(grads_tf_relu, grads_my_relu)

NameError: ignored